In [1]:
import pickle
import os
import numpy as np
import pandas as pd
import torch
import gpytorch
from matplotlib import pyplot as plt
import math

%matplotlib inline
%load_ext autoreload
%autoreload 2

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
dfs = pickle.load( open( "dfs.data", "rb" ) )

In [219]:
dfs_filled=[]

In [220]:
for i in range(len(dfs)):
    dfs_filled.append(dfs[i].fillna(method='ffill'))

In [171]:
for i in range(len(dfs)):
    dfs[i].index=range(len(dfs[i]))

In [173]:
sepsis_cases=[]
normal_cases=[]
for i in range(len(dfs)):
    if dfs[i].iloc[-1,-1]==1:
        sepsis_cases.append(i)
    else:
        normal_cases.append(i)

In [174]:
sepsis=([dfs[i] for i in sepsis_cases])

In [222]:
sepsis_f=([dfs_filled[i] for i in sepsis_cases])

In [175]:
normal=([dfs[i] for i in normal_cases])

In [223]:
normal_f=([dfs_filled[i] for i in normal_cases])

In [176]:
sepsis_times=[]
for i in range(len(sepsis)):
    sepsis_times.append(sepsis[i].iloc[:,-1].idxmax())

In [188]:
del sepsis_time_filter

In [253]:
sepsis_time_filter=[]
for i in range(len(sepsis_times)):
    if sepsis_times[i]>7:
        sepsis_time_filter.append(sepsis_f[i].iloc[(sepsis_times[i]-6):sepsis_times[i]+7,:])

In [239]:
sepsis_averages=[]
for i in range(len(sepsis)):
    sepsis_averages.append(sepsis[i].iloc[:,:34].mean(axis=None, skipna=True))

In [240]:
normal_averages=[]
for i in range(len(normal)):
    normal_averages.append(normal[i].iloc[:,:34].mean(axis=None, skipna=True))

In [242]:
type(sepsis_averages[0])

pandas.core.series.Series

In [246]:
norm_avg=pd.DataFrame([*normal_averages]).mean(axis=None, skipna=True)

In [248]:
seps_avg=pd.DataFrame([*sepsis_averages]).mean(axis=None, skipna=True)

In [250]:
seps_avg-norm_avg

HR                   5.708521
O2Sat                0.110202
Temp                 0.175606
SBP                 -2.237983
MAP                 -2.183010
DBP                 -1.825286
Resp                 1.191970
EtCO2               -1.037788
BaseExcess           0.070470
HCO3                -0.576158
FiO2                -0.032593
pH                   0.004743
PaCO2               -0.447264
SaO2                 0.286666
AST                 45.169681
BUN                  6.733423
Alkalinephos         2.172382
Calcium             -0.390526
Chloride             0.135230
Creatinine           0.280800
Bilirubin_direct     1.051411
Glucose              5.719396
Lactate              0.057328
Magnesium            0.032512
Phosphate            0.113513
Potassium           -0.001479
Bilirubin_total      0.671529
TroponinI            2.144151
Hct                 -1.177126
Hgb                 -0.409870
PTT                  2.826501
WBC                  1.846094
Fibrinogen          40.626188
Platelets 

In [255]:
sepsis_time_filter[3]

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,SepsisLabel
3,80.0,99.0,NaN,89,57.0,41.0,18,NaN,-3.0,21.0,0.5,7.37,36.0,88.0,NaN,34.0,NaN,NaN,112.0,NaN,NaN,129.0,NaN,NaN,NaN,4.3,NaN,NaN,32.8,11.4,NaN,25.1,NaN,NaN,77.26,0,0,1,-135.81,0
4,79.5,100.0,NaN,143,84.0,52.5,19,NaN,-3.0,21.0,0.5,7.30,47.0,88.0,NaN,34.0,NaN,NaN,112.0,NaN,NaN,177.0,NaN,NaN,NaN,4.3,NaN,NaN,33.6,11.4,NaN,25.1,NaN,NaN,77.26,0,0,1,-135.81,0
5,85.0,100.0,NaN,161,92.0,56.0,18,NaN,-3.0,21.0,0.5,7.30,47.0,88.0,NaN,34.0,NaN,NaN,112.0,NaN,NaN,177.0,NaN,NaN,NaN,4.3,NaN,NaN,33.6,11.4,NaN,25.1,NaN,NaN,77.26,0,0,1,-135.81,0
6,69.0,95.0,NaN,91,56.0,43.0,15,NaN,-3.0,21.0,0.5,7.30,47.0,88.0,NaN,34.0,NaN,NaN,112.0,NaN,NaN,177.0,NaN,NaN,NaN,4.3,NaN,NaN,33.6,11.4,NaN,25.1,NaN,NaN,77.26,0,0,1,-135.81,0
7,66.0,98.0,NaN,116,61.0,40.0,20,NaN,-2.0,21.0,0.5,7.30,46.0,71.0,NaN,34.0,NaN,NaN,112.0,NaN,NaN,177.0,NaN,NaN,NaN,4.3,NaN,NaN,33.6,11.4,NaN,25.1,NaN,NaN,77.26,0,0,1,-135.81,0
8,68.0,99.0,NaN,148,80.0,50.0,17,NaN,-2.0,21.0,0.5,7.32,46.0,71.0,NaN,34.0,NaN,NaN,112.0,NaN,NaN,177.0,NaN,NaN,NaN,4.3,NaN,NaN,33.6,11.4,NaN,25.1,NaN,NaN,77.26,0,0,1,-135.81,0
9,73.0,97.0,NaN,117,65.0,44.0,14,NaN,-2.0,21.0,0.5,7.32,46.0,71.0,NaN,34.0,NaN,NaN,112.0,NaN,NaN,177.0,NaN,NaN,NaN,4.3,NaN,NaN,33.6,11.4,NaN,25.1,NaN,NaN,77.26,0,0,1,-135.81,1
10,69.0,94.0,NaN,113,62.0,42.0,16,NaN,-2.0,23.0,0.5,7.32,46.0,71.0,NaN,27.0,NaN,NaN,110.0,0.7,NaN,106.0,NaN,2.4,NaN,4.9,NaN,NaN,27.3,9.7,NaN,20.2,NaN,148.0,77.26,0,0,1,-135.81,1
11,68.0,95.0,NaN,112,61.0,41.0,17,NaN,-2.0,23.0,0.5,7.34,44.0,96.0,NaN,27.0,NaN,NaN,110.0,0.7,NaN,106.0,NaN,2.4,NaN,4.9,NaN,NaN,27.3,9.7,NaN,20.2,NaN,148.0,77.26,0,0,1,-135.81,1
12,66.0,93.0,NaN,97,57.0,39.0,19,NaN,-2.0,23.0,0.5,7.34,44.0,81.0,NaN,27.0,NaN,NaN,110.0,0.7,NaN,106.0,NaN,2.4,NaN,4.9,NaN,NaN,27.3,9.7,NaN,20.2,NaN,148.0,77.26,0,0,1,-135.81,1


In [150]:
sepsis_times[3]

126

In [87]:
((sepsis_times>6))

,0
0,True
1,True
2,False
3,True
4,True
5,True
6,False
7,True
8,True
9,False


In [52]:
sepsis_times[i] for i in sepsis_times>5

TypeError: '>' not supported between instances of 'list' and 'int'

In [ ]:
df['Force'].lt(event).idxmax()

In [40]:
sepsis[1].iloc[:,-1].idxmax()

26

In [ ]:
len([dfs[i] for i in (sepsis_cases)])

In [ ]:
mx=0
for i in range(len(dfs)):
    if len(dfs[i])>mx:
        mx=i
print(mx)

In [ ]:
a=dfs[mx].iloc[:,:34]

In [ ]:
t=list()
for i in a.columns:
    t.append(a[i].dropna())

In [ ]:
x=list()
y=list()
columns=list()
for i in t:
    x.append(np.asarray(i.index))
    y.append(np.asarray(i))
    columns.append(i.name)

In [ ]:
missing=list()
for i in range(len(x)):
    if len(x[i])==0:
        missing.append(i)
        
for i in missing:
    del x[i]
    del y[i]
    del columns[i]

In [ ]:
x_torch=list()
y_torch=list()
for i in range(len(x)):
    x_torch.append(torch.from_numpy(x[i]).type(torch.FloatTensor))
    y_torch.append(torch.from_numpy(y[i]).type(torch.FloatTensor))

In [ ]:
class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.RBFKernel()

        # We learn an IndexKernel for 2 tasks
        # (so we'll actually learn 2x2=4 tasks with correlations)
        self.task_covar_module = gpytorch.kernels.IndexKernel(num_tasks=len(x), rank=len(x)-1)

    def forward(self,x,i):
        mean_x = self.mean_module(x)

        # Get input-input covariance
        covar_x = self.covar_module(x)
        # Get task-task covariance
        covar_i = self.task_covar_module(i)
        # Multiply the two together to get the covariance we want
        covar = covar_x.mul(covar_i)

        return gpytorch.distributions.MultivariateNormal(mean_x, covar)


In [ ]:
likelihood = gpytorch.likelihoods.GaussianLikelihood()

In [ ]:
train_i_task=list()
for i in range(len(x)):
    train_i_task.append(torch.full_like(x_torch[i], dtype=torch.long, fill_value=i))

In [ ]:
full_train_x = torch.cat(x_torch)
full_train_i = torch.cat(train_i_task)
full_train_y = torch.cat(y_torch)

In [ ]:
model = MultitaskGPModel((full_train_x, full_train_i), full_train_y, likelihood)

In [ ]:
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam([
    {'params': model.parameters()},  # Includes GaussianLikelihood parameters
], lr=0.1)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(500):
    optimizer.zero_grad()
    output = model(full_train_x, full_train_i)
    loss = -mll(output, full_train_y)
    loss.backward()
    print('Iter %d/500 - Loss: %.3f' % (i + 1, loss.item()))
    optimizer.step()

In [ ]:
model.eval()
likelihood.eval()
test_x = torch.linspace(0, len(a), len(a)+1)
test_i_task=list()
for i in range(len(x)):
    test_i_task.append(torch.full_like(test_x, dtype=torch.long, fill_value=i))

observed_pred=list()
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    for i in range(len(x)):
        observed_pred.append(likelihood(model(test_x, test_i_task[i])))

In [ ]:
def ax_plot(ax, train_y, train_x, rand_var, title):
    # Get lower and upper confidence bounds
    lower, upper = rand_var.confidence_region()
    # Plot training data as black stars
    ax.plot(train_x.detach().numpy(), train_y.detach().numpy(), 'k*')
    # Predictive mean as blue line
    ax.plot(test_x.detach().numpy(), rand_var.mean.detach().numpy(), 'b')
    # Shade in confidence
    ax.fill_between(test_x.detach().numpy(), lower.detach().numpy(), upper.detach().numpy(), alpha=0.5)
    ax.legend(['Observed Data', 'Mean', 'Confidence'])
    ax.set_title(title)

In [ ]:
f = plt.figure(figsize=(30,40))
ax_plots=list()
for i in range(len(x)):
    ax_plots.append(plt.subplot(len(x),1,i+1))
for i in range(len(x)):
    ax_plot(ax_plots[i], y_torch[i], x_torch[i], observed_pred[i], columns[i])

In [ ]:
 np.hstack([y_torch[0], observed_pred[0]])

In [ ]:
len(y_torch[0])

In [ ]:
torch.cat((y_torch[0], observed_pred[0]), 0)